In [2]:
import requests

In [3]:
requests.post("http://localhost:5000/api/model", json={})

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [1]:
def get_whois():
    return requests.get("https://hacker-news.firebaseio.com/v0/user/whoishiring/submitted.json").json()

In [2]:
def get_post(id):
    return requests.get(f"https://hacker-news.firebaseio.com/v0/item/{id}.json").json()

In [6]:
get_post(23379196)['title']

'Ask HN: Who is hiring? (June 2020)'

In [7]:
def is_hiring_post(post):
    return post.get("title", "").startswith("Ask HN: Who")

In [8]:
resp = get_post(23379196)

In [12]:
is_hiring_post(resp)

True

In [13]:
job_posts = resp["kids"]